# Guess a Number - SageMaker notebook

*By Michael Ludvig*

Import the required modules

In [ ]:
import os
import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from sagemaker_job.misc import get_execution_role, wait_for_s3_object

**Job configuration**

In [ ]:
job_name_prefix = 'rl-guessnumber'
instance_type = "ml.c5.xlarge"

**Find out AWS resources**

In [ ]:
# Figure out S3 bucket
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

# Figure out execution role
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

**RL Estimator** - here the training happens

In [ ]:
local_mode = False

if local_mode:
    instance_type = 'local'
    !/bin/bash ./common/setup.sh

estimator = RLEstimator(entry_point="train-coach.py",
                        source_dir="sagemaker_job",
                        dependencies=["gym_guess_number"],
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11.0',
                        framework=RLFramework.MXNET,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        hyperparameters = {
                            #"RLCOACH_PRESET": "preset-clippedppo",
                            #"rl.agent_params.algorithm.discount": 0.9,
                            #"rl.evaluation_steps:EnvironmentEpisodes": 8,
                            "improve_steps": 200000,
                            "save_model": 0,
                            # After Hyperparameter Tuning
                            #"rl.agent_params.algorithm.clip_likelihood_ratio_using_epsilon": 0.10529195900220492,
                            #"rl.agent_params.algorithm.discount": 0.9812742160193352,
                            #"rl.agent_params.algorithm.optimization_epochs": 7,
                        }
                    )

estimator.fit(wait=False)

**Figure out the reports names**

In [ ]:
job_name=estimator._current_job_name
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket,job_name)

if local_mode:
    output_tar_key = "{}/output.tar.gz".format(job_name)
else:
    output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

**Plot training progress**

In [ ]:
%matplotlib inline
import pandas as pd

csv_file_name = "worker_0.simple_rl_graph.main_level.main_level.agent_0.csv"
key = os.path.join(intermediate_folder_key, csv_file_name)
wait_for_s3_object(s3_bucket, key, tmp_dir, training_job_name=job_name)

csv_file = "{}/{}".format(tmp_dir, csv_file_name)
df = pd.read_csv(csv_file)
df = df.dropna(subset=['Training Reward'])

x_axis = 'Episode #'
y_axis = 'Training Reward'

df['group'] = df['Episode #'].floordiv(1000)
avg = df.groupby('group')['Training Reward'].mean()
plt = avg.plot()
plt.set_xlabel('Episode x1000')
plt.set_ylabel('Average Reward')